deep dive on AR modeling

*  explain ϕ parameter (0.9, -0.9)
*  use ACF to explain
* Compare the ACF for Several AR Time Series
*  Explain autoregressive formula
* estimating and forecasting
* analyzing results
* mean reverting
* pacf
* information criteria (AIC, BIC)